# Solid mechanics: Linear elasticity
---

## Introduction

In this first tutorial we

1.   Present a basic implementation of the finite element solution of Navier-Poisson problem
2.   Create a non-trivial geometry with the `gmsh` library
3.   Visualize the solution using pyvista and/or Paraview tools
4.   Perform some postprocessing


**Mathematical formulation:**

  ...

Finally, recall that the discrete version of this problem follows from applying the Galerkin method: Find $u_h \in V_{hg} \subset V_g(\Omega)$ such that

\begin{equation}
a(u_h,v_h) = \ell(v_h)~~ \forall v_h \in V_{h0} 
\end{equation}

## Initialization

The first step is to import all necessary libraries. In particular, we must import 
the [`FEniCSx`](https://fenicsproject.org/) library, which can be done now in Colab thanks to the efforts of the
[`FEM on Colab`](https://fem-on-colab.github.io/).
Notice that the first time the library is imported, the system may take a while. Following times are expected to be faster. 

In [1]:
try:
  import gmsh
except ImportError:
  !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/7f220b6/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
  import gmsh

try:
  import dolfinx
except ImportError:
  !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/7f220b6/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
  import dolfinx

In [14]:
from dolfinx import mesh, fem, io, plot
from ufl import SpatialCoordinate, TestFunction, TrialFunction, Measure, Identity, div, dx, ds, grad, nabla_grad, inner, sym, as_vector, FacetNormal

import numpy as np
from mpi4py import MPI
from petsc4py.PETSc import ScalarType

In [51]:
import gmsh
gmsh.initialize()

proc = MPI.COMM_WORLD.rank
if proc == 0:

    lc = 0.05

    Db   = 0.4
    Hb   = 0.4
    Hp   = 6*Hb
    R    = 3*Hb
    TT   = np.sqrt(R*R - 4*Hb*Hb)
    
    gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
    gmsh.model.geo.addPoint(Db, 0, 0, lc, 2)
    gmsh.model.geo.addPoint(Db, Hb, 0, 0.5*lc, 3)
    gmsh.model.geo.addPoint(TT+Db, 3*Hb, 0, lc, 4)
    gmsh.model.geo.addPoint(Db, 5*Hb, 0, lc, 5)
    gmsh.model.geo.addPoint(Db, 6*Hb, 0, 0.5*lc, 6)
    gmsh.model.geo.addPoint(0, 6*Hb, 0, lc, 7)
    gmsh.model.geo.addPoint(0, 3*Hb, 0, 0.1*lc, 8)
    gmsh.model.geo.addPoint(TT+Db-R, 3*Hb, 0, 0.1*lc, 9)
    
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)

    gmsh.model.geo.addCircleArc(3, 4, 9, 3)
    gmsh.model.geo.addCircleArc(9, 4, 5, 4)
    
    gmsh.model.geo.addLine(5, 6, 5)
    gmsh.model.geo.addLine(6, 7, 6)
    gmsh.model.geo.addLine(7, 8, 7)
    gmsh.model.geo.addLine(8, 1, 8)
    
    gmsh.model.geo.addCurveLoop([1, 2, 3, 4, 5, 6, 7, 8], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    # Tag the whole boundary with 101
    gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4, 5, 6, 7, 8], 101)
    # Tag the top boundary with 100
    gmsh.model.addPhysicalGroup(1, [6], 100)
    ps = gmsh.model.addPhysicalGroup(2, [1])
    gmsh.model.setPhysicalName(2, ps, "My surface") 
    gmsh.model.geo.synchronize()
    gmsh.option.setNumber("Mesh.Algorithm", 6)
    gmsh.model.mesh.generate(2)
    msh, subdomains, boundaries = io.gmshio.model_to_mesh(gmsh.model, comm=MPI.COMM_WORLD, rank=0, gdim=2)
    gmsh.finalize()
    #return msh, subdomains, boundaries

#msh, subdomains, boundaries = GenerateMesh()

with io.XDMFFile(MPI.COMM_WORLD, "body.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)

In [56]:
V = fem.VectorFunctionSpace(msh, ("CG", 1))

u, v = TrialFunction(V), TestFunction(V)

fdim = msh.topology.dim - 1

u_bottom = ScalarType((0.0, 0.0))
u_top    = ScalarType((0.0, -0.1))
u_left   = ScalarType(0.0)

# For the left boundary, just restrict u_x
facets_left = mesh.locate_entities_boundary(msh, fdim, lambda x: np.isclose(x[0], 0.0))
dofsL = fem.locate_dofs_topological(V.sub(0), fdim, facets_left)

# For the bottom restrict everything
dofsB = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], 0.0))
#dofsT = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], Hp))
#bcs = [fem.dirichletbc(u_bottom, dofsB, V), fem.dirichletbc(u_top, dofsT, V), fem.dirichletbc(u_left, dofsL, V.sub(0))]
bcs = [fem.dirichletbc(u_bottom, dofsB, V), fem.dirichletbc(u_left, dofsL, V.sub(0))]

# The rest of the boundary is traction free, except for the top in which we apply a surface force distribution

# surface force
F = fem.Constant(msh, ScalarType( (0.0, -0.1) ) )

# Body force
f = fem.Constant(msh, ScalarType( (0.0, 0.0) ) )

# Constitutive parameters
E, nu = 10.0, 0.3
mu    = E/(2.0*(1.0 + nu))
lmbda = E*nu/((1.0 + nu)*(1.0 - 2.0*nu))

def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

def sigma(u):
    return lmbda * div(u) * Identity(2) + 2*mu*epsilon(u)

x = SpatialCoordinate(msh)

ds = Measure("ds")(subdomain_data=boundaries)
one = fem.Constant(msh, 1.0)
length_form = fem.form( one*ds(100) )
lengthside = fem.assemble_scalar(length_form)
print(lengthside)

a = inner(sigma(u), epsilon(v)) * dx
L = inner(f, v) * dx + inner(F,v)*ds(100)

petsc_opts={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps", "ksp_monitor": None}
#petsc_opts={"ksp_type": "gmres", "ksp_rtol":1e-10, "ksp_atol":1e-10, "ksp_max_it": 1000, "pc_type": "hypre", "ksp_monitor": None}
problem = fem.petsc.LinearProblem(a, L, bcs=bcs, petsc_options=petsc_opts)
uh = problem.solve()
uh.name = "displacement"

# Save the results
with io.XDMFFile(MPI.COMM_WORLD, "displacement.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)
    xdmf.write_function(uh)


0.4
  Residual norms for dolfinx_solve_140238861361408 solve.
  0 KSP Residual norm 1.149050574825e-02 
  1 KSP Residual norm 5.929279755910e-15 
